In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import TimeDistributedDense, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from keras import backend as K

raw = open('wikigold.conll.txt', 'r').readlines()

all_x = []
point = []
for line in raw:
    stripped_line = line.strip().split(' ')
    point.append(stripped_line)
    if line == '\n':
        all_x.append(point[:-1])
        point = []
all_x = all_x[:-1]

Using Theano backend.
Using gpu device 0: GeForce GTX 960M (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5105)
/home/pritish/.local/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
/home/pritish/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
lengths = [len(x) for x in all_x]
print ('Input sequence length range: ', max(lengths), min(lengths))

short_x = [x for x in all_x if len(x) < 64]

X = [[c[0] for c in x] for x in short_x]
y = [[c[1] for c in y] for y in short_x]

all_text = [c for x in X for c in x]

Input sequence length range:  144 1


In [3]:
words = list(set(all_text))
word2ind = {word: index for index, word in enumerate(words)}
ind2word = {index: word for index, word in enumerate(words)}
labels = list(set([c for x in y for c in x]))
label2ind = {label: (index + 1) for index, label in enumerate(labels)}
ind2label = {(index + 1): label for index, label in enumerate(labels)}
print ('Vocabulary size:', len(word2ind), len(label2ind))

maxlen = max([len(x) for x in X])
print ('Maximum sequence length:', maxlen)
print(label2ind)

def encode(x, n):
    result = np.zeros(n)
    result[x] = 1
    return result

X_enc = [[word2ind[c] for c in x] for x in X]
max_label = max(label2ind.values()) + 1
y_enc = [[0] * (maxlen - len(ey)) + [label2ind[c] for c in ey] for ey in y]
y_enc = [[encode(c, max_label) for c in ey] for ey in y_enc]

X_enc = pad_sequences(X_enc, maxlen=maxlen)
y_enc = pad_sequences(y_enc, maxlen=maxlen)

Vocabulary size: 8285 5
Maximum sequence length: 63
{'I-PER': 3, 'I-ORG': 1, 'I-LOC': 4, 'I-MISC': 5, 'O': 2}


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size=11*32, train_size=45*32, random_state=42)
print ('Training and testing tensor shapes:', X_train.shape, X_test.shape, y_train.shape, y_test.shape)

max_features = len(word2ind)
embedding_size = 128
hidden_size = 32
out_size = len(label2ind) + 1

model = Sequential()
model.add(Embedding(max_features, embedding_size, input_length=maxlen, mask_zero=True))
model.add(LSTM(hidden_size, return_sequences=True))  
model.add(TimeDistributedDense(out_size))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()
batch_size = 32
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=10, validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, batch_size=batch_size)
print('Raw test score:', score)

Training and testing tensor shapes: (1440, 63) (352, 63) (1440, 63, 6) (352, 63, 6)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_1 (Embedding)          (None, 63, 128)       1060480     embedding_input_1[0][0]          
____________________________________________________________________________________________________


/usr/local/lib/python3.5/dist-packages/keras/layers/core.py:1112: UserWarning: TimeDistributedDense is deprecated, please use TimeDistributed(Dense(...)) instead.
  warnings.warn('TimeDistributedDense is deprecated, '


lstm_1 (LSTM)                    (None, 63, 32)        20608       embedding_1[0][0]                
____________________________________________________________________________________________________
timedistributeddense_1 (TimeDistr(None, 63, 6)         198         lstm_1[0][0]                     
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 63, 6)         0           timedistributeddense_1[0][0]     
Total params: 1081286
____________________________________________________________________________________________________
Train on 1440 samples, validate on 352 samples
Epoch 1/10
1440/1440 [==============================] - 1s - loss: 1.1916 - val_loss: 0.7767
Epoch 2/10
1440/1440 [==============================] - 1s - loss: 0.7023 - val_loss: 0.6362
Epoch 3/10
1440/1440 [==============================] - 1s - loss: 0.5650 - val_loss: 0.5199
Epoch 4/10
1440/1440 [========================

In [5]:
def score(yh, pr):
    coords = [np.where(yhh > 0)[0][0] for yhh in yh]
    yh = [yhh[co:] for yhh, co in zip(yh, coords)]
    ypr = [prr[co:] for prr, co in zip(pr, coords)]
    fyh = [c for row in yh for c in row]
    fpr = [c for row in ypr for c in row]
    return fyh, fpr

pr = model.predict_classes(X_train)
yh = y_train.argmax(2)
fyh, fpr = score(yh, pr)
print ('Training accuracy:', accuracy_score(fyh, fpr))
print ('Training confusion matrix:')
print (confusion_matrix(fyh, fpr))
precision_recall_fscore_support(fyh, fpr)

pr = model.predict_classes(X_test)
yh = y_test.argmax(2)
fyh, fpr = score(yh, pr)
print ('Testing accuracy:', accuracy_score(fyh, fpr))
print ('Testing confusion matrix:')
print (confusion_matrix(fyh, fpr))
precision_recall_fscore_support(fyh, fpr)

1440/1440 [==============================] - 0s     
Training accuracy: 0.98590504451
Training confusion matrix:
[[ 1465    58     0    10     9]
 [   24 24708     0     9    11]
 [   11    17  1172    20     6]
 [   25    24    10  1055     2]
 [   21   138    12    11   838]]
352/352 [==============================] - 0s     
Testing accuracy: 0.899255247123
Testing confusion matrix:
[[ 205  103    8   14   27]
 [  85 6068   15    9   27]
 [  35   79  150    8   11]
 [  52   67    7  132   10]
 [  29  144    6    8   86]]


(array([ 0.50492611,  0.9391735 ,  0.80645161,  0.77192982,  0.53416149]),
 array([ 0.57422969,  0.97807866,  0.53003534,  0.49253731,  0.31501832]),
 array([ 0.53735256,  0.95823135,  0.63965885,  0.60136674,  0.39631336]),
 array([ 357, 6204,  283,  268,  273]))

In [9]:
string = "John was a member of US Army"
wordlist = string.split(' ')
ip = []
for x in wordlist:
    ip.append(word2ind[x])
i=maxlen-len(ip)
temp=[0]*i
ip=temp+ip
op = K.function([model.layers[0].input], model.layers[3].output)
out = op([[ip]])
temp = []
while i<maxlen:
    for j in label2ind:
        if label2ind[j]==out[0][i].tolist().index(max(out[0][i])):
            temp.append(j)
    i=i+1
print(wordlist)
print(temp)

['John', 'was', 'a', 'member', 'of', 'US', 'Army']
['I-PER', 'O', 'O', 'O', 'O', 'I-LOC', 'I-ORG']
